In [60]:
import numpy as np
import glob
from pathlib import Path
import json
from multiprocessing import Pool
from tqdm import tqdm

## global setup

In [61]:
mymodel_dir = "/data/home/yanghanlong/results/image_align/random_5k_0707/v2_2_epoch3"
orimodel_dir = "/data/home/yanghanlong/results/image_align/random_5k_0707/original_model"


In [62]:
mymodel_results = [Path(x).name for x in glob.iglob(f"{mymodel_dir}/*.json")]
orimodel_results = [Path(x).name for x in glob.iglob(f"{orimodel_dir}/*.json")]
common_results = list(set(mymodel_results) & set(orimodel_results))

## some base functions

In [63]:
def transform_template_pt(template_pt, H, tps_weights, template_mkpts_tps_,
                            parallex, overlap, K_smooth = 5):
    '''
    Args:
        template_pt: ndarray, (2, ),
        template_mkpts_tps_: ndarray, (2, n),
        parallex: ndarray, (2, n)
    '''
    ## Do global homographic transform
    template_hpt = np.append(template_pt, 1) # (3, )
    # import pdb
    # pdb.set_trace()
    template_hpt_ = H @ template_hpt # (3, )
    template_pt_ = template_hpt_ / (template_hpt_[2] + 1e-8)
    u_, v_ = template_pt_[:2]

    ## Do tps transformation
    gx, hy = 0, 0
    offset_u0_, offset_u1_ = overlap['offset_u0_'], overlap['offset_u1_']
    offset_v0_, offset_v1_ = overlap['offset_v0_'], overlap['offset_v1_']
    if (offset_u0_ <= u_ <= offset_u1_) and (offset_v0_ <= v_ <= offset_v1_):
        x1_, y1_ = template_mkpts_tps_
        n = len(x1_)
        gx_sub, hy_sub = 0, 0
        wx, wy = tps_weights['wx'], tps_weights['wy']
        a, b = tps_weights['a'], tps_weights['b']
        for kf in range(n):
            dist2 = (u_ - x1_[kf]) ** 2 + (v_ - y1_[kf]) ** 2
            rbf = 0.5 * dist2 * np.log(dist2)
            gx_sub += wx[kf] * rbf
            hy_sub += wy[kf] * rbf

        gx_sub += a[0] * u_ + a[1] * v_ + a[2]
        hy_sub += b[0] * u_ + b[1] * v_ + b[2]
        gx, hy = gx_sub, hy_sub

    ## smooth transition to global transform
    gxn, hyn = parallex
    eta_d0 = 0
    eta_d1 = K_smooth * max(abs(np.concatenate([gxn, hyn])))
    sub_u0_, sub_u1_ = overlap['sub_u0_'], overlap['sub_u1_']
    sub_v0_, sub_v1_ = overlap['sub_v0_'], overlap['sub_v1_']
    sub_u0_ = sub_u0_ + min(gxn)
    sub_u1_ = sub_u1_ + max(gxn)
    sub_v0_ = sub_v0_ + min(hyn)
    sub_v1_ = sub_v1_ + max(hyn)
    dist_horizontal = np.maximum(sub_u0_-u_, u_-sub_u1_)
    dist_vertical = np.maximum(sub_v0_-v_, v_-sub_v1_)
    dist_sub = np.maximum(dist_horizontal, dist_vertical)
    dist_sub = np.maximum(0, dist_sub)
    eta = (eta_d1 - dist_sub) / (eta_d1 - eta_d0)
    # eta[dist_sub < eta_d0] = 1
    # eta[dist_sub > eta_d1] = 0
    if dist_sub < eta_d0:
        eta = 1
    elif dist_sub > eta_d1:
        eta = 0

    gx *= eta
    hy *= eta

    res_u_ = u_ - gx
    res_v_ = v_ - hy
    return [res_u_, res_v_]

def findPossibleGTIndex(data):
    index = -1; mpts = 0
    if data["alignment_results"] == []:
        return index
    for k,v in data["alignment_results"].items():
        if v["matched_features"]["mkpts_count"] > mpts:
            index = k
            mpts = v["matched_features"]["mkpts_count"]
    return index

def find_mpt(tem_pt, alignment_params):

    H, tps_weights, template_mkpts_tps_,parallex, overlap = alignment_params["H"],alignment_params["tps_weights"],alignment_params["template_mkpts_tps_"],alignment_params["parallex"],alignment_params["overlap"]
    sam_pt = transform_template_pt(tem_pt, H, tps_weights, template_mkpts_tps_,parallex, overlap, K_smooth = 5)
    return (tem_pt, sam_pt) # ([x1,x2],[y1,y2])

def sample_10_by_10(alignment_params,new_shape): # return: [(tem_pt,sam_pt),(tem_pt,sam_pt)]
    # sample 10x10 pts from template (e.p. for 480x640 size: 0,...,480,0,...,640)
    new_shape = np.array(new_shape)
    x = np.linspace(0,new_shape[0],num=10)
    y = np.linspace(0,new_shape[1],num=10)
    x,y = np.meshgrid(x,y)
    template_pts = np.concatenate((x.reshape(-1, 1), y.reshape(-1, 1)), axis=1) # 100x2
    return [find_mpt(x, alignment_params) for x in template_pts]

def readResults(js):
    with open(js, 'r') as f:
        data = json.load(f)
    gt_index = findPossibleGTIndex(data)
    if gt_index == -1:
        return -1,-1,-1
    # find gt alignment results
    alignment_params = data["alignment_results"][gt_index]["alignment_params"]
    template_path = data["alignment_results"][gt_index]["template_path"]
    new_shape = data["alignment_results"][gt_index]["template_features"]["resized_tem_shape"][::-1][1:]
    return alignment_params, template_path,new_shape


## compare single _[only for 960 compared to 480]_

In [79]:
def compare_single(fname):
    # open original results
    ori_alignment_params, ori_template_path,ori_new_shape = readResults(f"{orimodel_dir}/{fname}") #new_shape:[w,h]
    
    # open mymodel results
    my_alignment_params, my_template_path,my_new_shape = readResults(f"{mymodel_dir}/{fname}")
    if my_alignment_params == -1 and ori_alignment_params != -1:
        print(f"[My model img align failure] {fname}")
        return ("My model img align failure",Path(fname).stem)
    elif ori_alignment_params == -1 and my_alignment_params != -1:
        print(f"[Original model img align failure] {fname}")
        return ("Original model img align failure",Path(fname).stem)
    elif ori_alignment_params == -1 and my_alignment_params == -1:
        return ("All img align failure(most likely not GT)",Path(fname).stem)
    # check if Most possible GT consistent
    if ori_template_path != my_template_path:
        print(f"[Most possible GT inconsistent] {fname}\nori_template_path:{ori_template_path}\nmy_template_path:{my_template_path}")
        return ("Most possible not consistent",Path(fname).stem)
    scale_factor = my_new_shape[0] / ori_new_shape[0]

    # original pts and my model pts
    original_ptmap = sample_10_by_10(ori_alignment_params,ori_new_shape) # return: [(tem_pt,sam_pt),(tem_pt,sam_pt)]
    my_ptmap = sample_10_by_10(my_alignment_params,my_new_shape)
    
    ##calculate distance
    distance = [np.linalg.norm(np.array(x[1]) - np.array(y[1]) / scale_factor) for x,y in zip(original_ptmap, my_ptmap)] # normalize to (480,640)
    aver_distance = sum(distance) / len(distance)
    return (aver_distance.item(),Path(fname).stem)
##debug
# common_results[0:5]
# compare_single("2700920e-d641-4672-b8ec-1b582d4e36cb_result.json")

## run by mp

In [80]:
with Pool(32) as p:
   distances = p.map(compare_single,tqdm(common_results))

  0%|          | 0/4702 [00:00<?, ?it/s]

[Original model img align failure] 7b3a2a6d-5433-4b0a-87aa-598b0e85abde_result.json
[My model img align failure] d25af30f-338b-4f61-8d35-6845c0649645_result.json
[My model img align failure] 58c5d094-c855-4245-a0c2-f641adce0806_result.json
[My model img align failure] 725e7747-188a-459a-b1b6-b71f21b1a7df_result.json


 54%|█████▎    | 2516/4702 [00:19<00:16, 133.48it/s]

[My model img align failure] 97612017-b9fb-4adf-b047-b4e30a4d10ea_result.json


 55%|█████▌    | 2590/4702 [00:19<00:15, 137.21it/s]

[My model img align failure] f834abbd-d913-4e57-931d-0c4b1148a24e_result.json


 74%|███████▍  | 3478/4702 [00:23<00:08, 141.46it/s]

[My model img align failure] 9d18881f-b86e-4129-8f3e-6a688f1e2717_result.json
[My model img align failure] 8d53b866-3be5-4861-b493-17ddee5bd440_result.json
[My model img align failure] 0ffc0108-840f-4c63-9039-0df496cae0bc_result.json
[My model img align failure] 3fd34b3c-963d-4536-bab1-b53adf0ebc67_result.json


100%|██████████| 4702/4702 [00:44<00:00, 106.20it/s]


[Most possible GT inconsistent] f687b676-8475-4690-9f56-f0b566210e9e_result.json
ori_template_path:8905500586069871pic.jpg
my_template_path:5175115207774969pic.jpg
[My model img align failure] e4e2136e-c709-45e5-a442-2183378c74b9_result.json
[Most possible GT inconsistent] d30f2fb3-addd-48ff-9ea9-00034b3a8b6f_result.json
ori_template_path:5175119195147937pic.jpg
my_template_path:4046405146310375pic.jpg
[Original model img align failure] d58ee5b6-2133-4d9e-b5c3-1ef209146500_result.json
[Most possible GT inconsistent] 4e1a8faa-2974-4d0d-99e3-87df6b0ddb9c_result.json
ori_template_path:3106557370655908pic.jpg
my_template_path:4046403605989732pic.jpg
[Original model img align failure] 00c9b431-633e-4f0c-91d4-3c8d1af9939f_result.json
[Original model img align failure] d2b2e475-c1fe-4cff-bf69-33fd0fc50730_result.json
[Original model img align failure] 6ded5fbf-f98e-4693-ac4a-69425fbb35aa_result.json
[My model img align failure] df2d1969-fe8f-413e-a8ca-850c3e039173_result.json
[Most possible G

## analysis 

In [81]:
cleaned_distances = [x for x in distances if type(x[0]) != str]

In [82]:
sorted(cleaned_distances,reverse=True,key=lambda x:x[0])

[(10800.99924919475, 'bb54a24d-d73a-4554-bc7c-3e008226e02c_result'),
 (8947.71614591498, 'defa09de-e94a-4254-b528-395e2320ac2f_result'),
 (3155.0159803191345, '3953d2bf-d4a7-4d82-b6e7-9827c8e57ee7_result'),
 (562.8809876004648, '6d973435-814a-4a14-be01-7a622d659536_result'),
 (112.37286181488747, 'd5cdd99b-e409-4368-bb1a-8783a2175c7b_result'),
 (90.80308510671244, 'c318daf1-65fb-43f3-b18c-b3d60263d972_result'),
 (80.1961027845091, '29bb2316-dd47-421e-8985-683fabaa32e6_result'),
 (63.264979539631405, 'd03e589f-364d-44ee-85ad-524c3e64db6a_result'),
 (57.02078996761878, 'cb14404d-053a-4d1b-bb4e-814a2094439d_result'),
 (56.64452263712682, '2ee12cbb-77fb-41e6-8b40-11ba659fd83c_result'),
 (55.54255803393858, '8b04eba3-8fd6-499e-a7b7-20e25ae78874_result'),
 (54.5896902843153, '6d87db18-eaed-4b99-bd26-ed840c069621_result'),
 (46.31945584573246, 'fce2b12a-bde8-4b8c-906e-b30d5cf5799c_result'),
 (42.68663484725838, '447529e6-ed8c-45b6-aedb-8173282f4ccd_result'),
 (42.55393340824617, '0cabf582-957

da61d0ae-9152-45ec-906b-08844ce25c38